# Magnetic Monte Carlo with CINOLA

This workflow makes use of a spin dynamics code named CINOLA, which is developed in-house at Hochschule Bielefeld University of Applied Sciences and Arts. Head developer and original author is Prof. Dr. Christian Schröder.

The spin dynamics code is integrated via a call to the compiled binary. This workflow takes of care of the required input and output format of the spin dynamics code.

In [ ]:
import cinola_interface as cin

In [ ]:
import numpy as np
import matplotlib.pylab as plt
%config InlineBackend.figure_format = 'retina'

In [ ]:
from pathlib import Path
import shutil


# boiler plate for pyiron in local development environment

To use this workflow in a productive pyiron environment,

1. remove this cell,
2. place the files of the dynamic pyiron job (`./pyiron/resources/dynamic/MyCinolaJob/`) into the correct ressource path of your pyiron installation,
3. install the cinola_interface module (`./cinola_interface.py`) in your Python environment.

> Yes, this is still a process with manual steps. New versions will clean this up.

The pyiron CINOLA job is for dev purposes located inside this project, not the central user folder. It is in ./pyiron/resources/

In [ ]:
# %%
root = Path('.')

# %%
# Removing files from previous tests

try:
    shutil.rmtree(str((root / 'pyiron-run').resolve()))
except FileNotFoundError:
    pass
try:
    (root / 'info.log').unlink(missing_ok=True)
    (root / 'pyiron.db').unlink(missing_ok=True)
    (root / 'pyiron.log').unlink(missing_ok=True)
except PermissionError:
    print('This does only work, if pyiron is not loaded already. Else it has exclusive access to this file.')

# %%
import pyiron
import pyiron_base
import pyiron_atomistics
print('pyiron: ' + pyiron.__version__)
print('pyiron_base: ' + pyiron_base.__version__)
print('pyiron_atomistics: ' + pyiron_atomistics.__version__)

# %%
import sys
for idx, path in enumerate(sys.path, 1):
    print(f'{idx} - {path}')
print('\n')
print('pyiron: ' + pyiron.__file__)
print('pyiron_base: ' + pyiron_base.__file__)
print('pyiron_atomistics: ' + pyiron_atomistics.__file__)


# %%
from pyiron import Project
from pyiron_base.generic.dynamic import _get_template_classes
pr = Project(path='pyiron-run')
pr.state.update({
    'sql_file': './pyiron.db',
    'sql_type': 'SQLite',
    'resource_paths': str(root / 'pyiron' / 'resources'),
})
pr.create.job._job_dyn_dict = _get_template_classes()

pr.remove_jobs(recursive=True, silently=True)

# Preparing $J_{}$

In [ ]:
def get_J_in_K_from_eV(J_eV):
    boltzmann_eVperK = 8.617e-5 #* ureg.boltzmann_constant
    return (J_eV / (boltzmann_eVperK))

## Input of J and structure parameters

Modify this to your liking.

This is example data taken from a paper on Gd: R Essajai et al 2021 Phys. Scr. 96 015808 https://doi.org/10.1088/1402-4896/abc984

Later, the user has to put in configuration for the structure usign pyiron measures. 

The Js are expected to be in eV. The list of Js contains all Js, starting from the inner-most shell, then continuing up to the outer-most shell. What inner- and outer-most shell means is based on the distances pyiron calculates and the sorting it then does in its structure / neighborhood attributes.

In [ ]:
a, c = 3.629, 5.796
structure = pr.create.structure.bulk('Gd', a=a, c=c, crystalstructure='hcp' ).repeat(8)
Js = [2.55e-3, 1.74e-3] # eV

# Another test with fcc Ni with a=3.532 Angstrom and repeat(18)
structure = pr.create.structure.bulk('Ni', a=3.532, crystalstructure='fcc' ).repeat(18)
Js = [7.13245e-3, 12.73309e-3] # eV


In [ ]:
#structure = pr.create.structure.bulk('Fe', cubic=True).repeat(3)
#Js = [.0167] #eV

In [ ]:
Js

## Unusual scaling due to the article, we used as reference for the exchange parameters

This scaling is not usual. However, it makes the important point that it has to be thoroughly described what J actually means here and in general! 

In [ ]:
Js = [J / 6 for J in Js]

In [ ]:
Js

## Scaling of J with mu_eff

This is required by the simulation code.

Adjust the $\mu_{eff}$ here.

CINOLA assumes J scaled with $\mu_{eff}$. The scaling is done here. Therefor, $\mu_{eff}$ *mu_eff_prefactor* has to be put in by the user here.

In [ ]:
mu_eff_prefactor = 7.22


In [ ]:
#muB = 9.274009e-24; # J/T
muB = 5.788382e-5; # eV/T
kB_JperK = 1.380e-23 # J/K
kB_eVperK = 8.617e-5 # eV/K

In [ ]:
mu_eff = mu_eff_prefactor * muB / kB_eVperK

In [ ]:

Js = [J * mu_eff ** 2 for J in Js]


In [ ]:
Js

## Converting J[eV] into J[K] for CINOLA

In [ ]:
Js_K = [get_J_in_K_from_eV(J) for J in Js]

In [ ]:
Js_K

# Create $J_{ij}$ from structure and input Js

In [ ]:
structure.plot3d()

In [ ]:
len(structure)

In [ ]:
_, Jij = cin.get_neighborhoods(structure=structure, Js_K=Js_K)

# Additional pyiron structure setup

In [ ]:
structure.set_initial_magnetic_moments([mu_eff_prefactor]*len(structure))


# Run CINOLA with pyiron

Choose the number of iterations per temperature here:

Please input the number of MC iterations you would like to to per temperature here.

In [ ]:
iterations_per_temperature_cinola = 1000_000#00

`delete_exisiting_job=True` is primarily used in the development of this workflow. Please adjust as needed for your usecase.

In [ ]:
job = pr.create.job.MyCinolaJob('script', delete_existing_job=True)

## Additional inputs

Adjust here:

1. `H_value`: The external magnetic field in direction of XXX (to be filled). The direction is not yet exposed to the workflow.
2. `T_low`: Minimum of the simulated temperature sweep.
3. `T_high`: Maximum of the simulated temperature sweep.
4. `T_step`: Step size of the simulated temperature sweep. Can be positive or negative.

In [ ]:
job.input['H_value'] = 1
job.input['T_low'] = 200
job.input['T_high'] = 350
job.input['T_step'] = 5

In [ ]:
job.input['structure'] = structure
job.input['Js_K'] = Js_K
job.input['num_iter_per_temp'] = iterations_per_temperature_cinola


In [ ]:
# Do it
job.run()

# Extract results

In [ ]:
B_cinola = job.output['B_dfs'][0]['B']

# Compare with Sam's studio MC

https://github.com/samwaseda/sams-studio/

In [ ]:
iterations_per_temperature_sams = 1000#_00

In [ ]:
from samsstudio.MC.mc import MC

In [ ]:
boltzmann_eVperK = 8.617e-5


In [ ]:
mc = MC(len(structure))
mc.set_heisenberg_coeff(Jij * boltzmann_eVperK)



In [ ]:
def magnetization_on_temperature(temperature):
    mc.run(temperature=temperature, number_of_iterations=iterations_per_temperature_sams)
    return mc.get_output()['magnetization']

In [ ]:
sams_T = job.output['B_dfs'][0]['df']['T_[K]']
sams_M = [magnetization_on_temperature(temperature) for temperature in sams_T]

## Comparison

In [ ]:
from datetime import datetime

In [ ]:
job.output['B_dfs'][0]['df'][['T_[K]', 'MagAvgMag_[mu_B]']].plot(x='T_[K]', y='MagAvgMag_[mu_B]', label=f'CINOLA ({iterations_per_temperature_cinola:.1e} iterations, B={float(B_cinola):.1e} T)')
plt.plot(sams_T, sams_M, label=f"Sam's ({iterations_per_temperature_sams:.1e} iterations)")
plt.legend()
plt.ylabel('Average Magnetization Magnitude $[\mu_B]$')
plt.title(datetime.now())


In [ ]:
x_iend = 15
job.output['B_dfs'][0]['df'][['T_[K]', 'MagAvgMag_[mu_B]']][:x_iend].plot(x='T_[K]', y='MagAvgMag_[mu_B]', label=f'CINOLA ({iterations_per_temperature_cinola:.1e} iterations, B={float(B_cinola):.1e} T)')
plt.plot(sams_T[:x_iend], sams_M[:x_iend], label=f"Sam's ({iterations_per_temperature_sams:.1e} iterations)")
plt.legend()
plt.ylabel('Average Magnetization Magnitude $[\mu_B]$')
plt.title(datetime.now())


## Compare Average Magnetization Magnitude and Magnetization projected on H

In [ ]:

plt.plot(sams_T, sams_M, label=f"Sam's ({iterations_per_temperature_sams:.1e} iterations)")
axplt = plt.gca()
axplt.legend()
axpd = job.output['B_dfs'][0]['df'][['T_[K]', 'MagAvgMag_[mu_B]']].plot(ax=axplt, x='T_[K]', y='MagAvgMag_[mu_B]', label=f'CINOLA ({iterations_per_temperature_cinola:.1e} iterations, B={float(B_cinola):.1e} T)')
plt.ylabel('Average Magnetization Magnitude')

axsec = job.output['B_dfs'][0]['df'][['T_[K]', 'MagHProjected_[mu_B]']].plot(ax=axpd, x='T_[K]', y='MagHProjected_[mu_B]', label=f'CINOLA ({iterations_per_temperature_cinola:.1e} iterations, B={float(B_cinola):.1e} T)', secondary_y=True)
plt.ylabel("Magnetization projected on H")

plt.title(datetime.now())

ybot_plt, ytop_plt = axplt.get_ylim()
ybot_pd, ytop_pd = axsec.get_ylim()
yheight_plt = ytop_plt - ybot_plt
yheight_pd = ytop_pd - ybot_pd
axplt.set_ylim(ybot_plt , ybot_plt+yheight_plt*1.3)
axsec.set_ylim(ybot_pd, ybot_pd+yheight_pd*1.3)

axplt.grid(ls='--')

In [ ]:

plt.plot(sams_T, sams_M, label=f"Sam's ({iterations_per_temperature_sams:.1e} iterations)")
axplt = plt.gca()
axplt.legend()
axpd = job.output['B_dfs'][0]['df'][['T_[K]', 'MagAvgMag_[mu_B]']].plot(ax=axplt, x='T_[K]', y='MagAvgMag_[mu_B]', label=f'CINOLA ({iterations_per_temperature_cinola:.1e} iterations, B={float(B_cinola):.1e} T)')
plt.ylabel('Average Magnetization Magnitude')

axsec = job.output['B_dfs'][0]['df'][['T_[K]', 'MagHProjected_[mu_B]']].plot(ax=axpd, x='T_[K]', y='MagHProjected_[mu_B]', label=f'CINOLA ({iterations_per_temperature_cinola:.1e} iterations, B={float(B_cinola):.1e} T)', secondary_y=True)
plt.ylabel("Magnetization projected on H")

plt.title(datetime.now())

ybot_plt, ytop_plt = axplt.get_ylim()
ybot_pd, ytop_pd = axsec.get_ylim()
yheight_plt = ytop_plt - ybot_plt
yheight_pd = ytop_pd - ybot_pd
axplt.set_ylim(ybot_plt , ybot_plt+yheight_plt*1.3)
axsec.set_ylim(ybot_pd, ybot_pd+yheight_pd*1.3)
axplt.set_xlim(-10, 500)
axplt.set_xlim(-10, 500)

axplt.grid(ls='--')

# Plot Susceptibility

In [ ]:
chiT_df = job.output['B_dfs'][0]['df'][['T_[K]', 'Chi*T(cgs)_[cm^3/mol]']]
chiT_df['chi'] = job.output['B_dfs'][0]['df']['Chi*T(cgs)_[cm^3/mol]'] / job.output['B_dfs'][0]['df']['T_[K]']
chiT_df.drop(0, inplace=True)
chiT_df.plot(x='T_[K]', y='chi', title='Chi*T(cgs)_[cm^3/mol] / T_[K]')
ax = plt.gca()
ax.grid(ls='--')